In [22]:
# imports
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as c
from matplotlib import rcParams
import sympy as sy
import scipy.constants as pc
from sympy import *
from math import log10, floor


In [24]:
# Funktionen
def am(werte):
    return np.mean(werte)

def std(werte):
    summen_der_quadrate = 0
    for wert in werte:
        summen_der_quadrate+=(wert-am(werte))**2       
    return np.sqrt(summen_der_quadrate/(werte.size-1))

def stu(werte):
    return std(werte)/np.sqrt(werte.size)

def fehler(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += (dif * unsicherheiten[i])**2
    zMax = sqrt(zMax)
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def fehlerMax(f, werte, unsicherheiten, variablen):
    zMax = 0
    for i in range(werte.size):
        dif = np.abs(sy.diff(f, variablen[i]))
        zMax += dif * unsicherheiten[i]
    for j in range(werte.size):
        zMax = sub(zMax, variablen[j], werte[j])
    return zMax

def groessterBeitrag(f, werte, unsicherheiten, variablen):
    zMax = 0
    zMaxArray = np.array([])
    for i in range (werte.size):
        zMax = (np.abs(sy.diff(f, variablen[i])) * unsicherheiten[i])
        for j in range(werte.size):
            zMax = sub(zMax, variablen[j], werte[j])
        x = np.array([zMax])
        zMaxArray = np.append(zMaxArray, x)
    return variablen[np.argmax(zMaxArray)], zMaxArray[np.argmax(zMaxArray)]

def sub(zMax, variable, wert):
    return zMax.subs([(variable, wert)])

def einfachGaussAdd(unsicherheiten, koeffizienten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (koeffizienten[i]*unsicherheiten[i])**2
    return sqrt(dz)

def einfachGaussMult(unsicherheiten, werte, exponenten):
    dz = 0
    for i in range (unsicherheiten.size):
        dz += (exponenten[i]*unsicherheiten[i]/werte[i])**2
    return sqrt(dz)

def einfachGaussMultMax(unsicherheiten, werte, exponenten):
    max_a = 0
    num = -1
    for i in range (unsicherheiten.size):
        maxi = (exponenten[i]*unsicherheiten[i]/werte[i])**2
        if max_a <= maxi:
            max_a = maxi
            num = i
    return (max_a, num)

def vertraeglichkeit(am_1, am_2, stu_1, stu_2):
    return (np.abs(am_1-am_2)/(sqrt((stu_1**2)*(stu_2**2))))

def bestA(x, y):
    return (sum(x**2)*sum(y)-sum(x)*sum(x*y))/(x.size*sum(x**2)-sum(x)**2)

def bestB(x, y):
    return (x.size*sum(x*y)-sum(x)*sum(y))/(x.size*sum(x**2)-sum(x)**2)

def streuung(x, y):
    s = 0
    for i in range(x.size):
        s += (y[i]-(bestA(x, y)+bestB(x, y)*x[i]))**2
    return np.sqrt(s/(x.size-2))

def stuA(x, y):
    s = streuung(x, y)
    return s*np.sqrt(sum(x**2)/((x.size*sum(x**2)-sum(x)**2)))

def stuB(x, y):
    s = streuung(x, y)
    return s*np.sqrt(x.size/((x.size*sum(x**2)-sum(x)**2)))

def round_to_j(x,j):
    return round(x, (j-1)-int(floor(log10(abs(x)))))

def print_to_table(avgs, stds, rows, cols,
                   highlight_best=True,
                   max_is_best=True,
                   avg_format='{0:!>4.1f}',
                   std_format='{1:!<4.1f}',
                   file=None,
                   col_orientation='c'):
    # Find best values
    best_avg = {}
    if highlight_best:
        if max_is_best:
            for c in cols:
                idx = np.argmax(avgs[c], axis=0)
                best_avg[c] = idx
        else:
            for c in cols:
                idx = np.argmin(avgs[c], axis=0)
                best_avg[c] = idx

    # Header
    print_str = '\\begin{tabular}{l' + (" "+col_orientation) * len(cols) + '} \hline\n'
    for c in cols:
        print_str += ' & ' + c
    print_str += ' \\\\ \n \hline'

    # Content
    for m in range(len(rows)):
        row_name = rows[m]
        print_str += '\n{0}'.format(row_name)
        for c in cols:
            print_str += ' &\n '
            print_list = [avgs[c][m]]
            print_list += [stds[c][m]]
            if m == best_avg.get(c, None):
                print_str += ('$\mathbf{{'+ avg_format + ' \\pm ' + std_format + '}}$').format(*print_list)
            else:
                print_str += ('$'+ avg_format + ' \\pm ' + std_format + '$').format(*print_list)

        print_str += "\n \\vspace*{0.0mm} \\\\"
        print_str = print_str.replace('!', '\\phantom{0}')

    # Footer
    print_str += '\n \hline \n \end{tabular}'

    # Write output
    if file is None:
        print(print_str)
    else:
        with open(file, 'w') as f:
            f.write(print_str)
            
    
def txtToArr(name):
    with open(name) as f:
        l = f.readlines()
    
    del l[0]

    time_arr = np.array([])
    temp_arr = np.array([])

    for el in l:
        el_changed = l[l.index(el)].replace(',', '.') 
        l[l.index(el)] = el_changed.split()
        el_changed = el_changed.split()
        time_arr = np.append(time_arr, float(el_changed[0]))
        temp_arr = np.append(temp_arr, float(el_changed[1]))
    
    return time_arr, temp_arr

In [ ]:
# Standard plotting

fig, ax = plt.subplots(figsize=(10,6))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

#ax.plot(x, y, color='b', label='label')
#ax.scatter(x, y, color='r', label='label')
#ax.errorbar(am_x, am_y, xerr=std_x, yerr=std_y, fmt='D', color='#d0740b', label='label')

ax.set(xlabel='x', ylabel='y', title='Titel')
ax.legend()
ax.grid()

#fig.savefig("test.png")

In [ ]:
# Erinnerungen
# x = np.array([,])
# x.size
# x, y = symbols('x y')
# expr = x*y
# expr.subs([(x, wert_x), (y, wert_y)])

In [ ]:
# KALIBRIERDATEN
temp_f = 0.1
time_f = 0.01

time_w_k, temp_w_k  = txtToArr('kochend.txt')
time_e_k, temp_e_k  = txtToArr('Eis.txt')


fig, (ax0, ax1) = plt.subplots(2, figsize=(12,5), sharex=True) 
ax0.tick_params(axis='both', which='major', labelsize=18)
ax0.tick_params(axis='both', which='minor', labelsize=16)
ax1.tick_params(axis='both', which='major', labelsize=18)
ax1.tick_params(axis='both', which='minor', labelsize=16)

ax0.errorbar(time_w_k, temp_w_k, xerr=time_f, yerr=temp_f, fmt='.', color='#ffd166', label=r'Messwerte $T_W$')

x_w = np.linspace(0, max(time_w_k)+1, 100)
y_w = np.full( shape=100, fill_value=am(temp_w_k))
ax0.plot(x_w, y_w, color='#ef476f', label='$\overline{T}_W$')


ax0.set(xlabel=r'', ylabel=r'$T_W$ [°C]', title=r'Temperatur von kochendem Wasser im Verlauf der Zeit')
ax0.legend()
ax0.grid()

ax1.errorbar(time_e_k, temp_e_k, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label=r'Messwerte $T_E$')

x_e = np.linspace(0, max(time_e_k)+1, 100)
y_e = np.full( shape=100, fill_value=am(temp_e_k))
ax1.plot(x_e, y_e, color='#073b4c', label='$\overline{T}_E$')


ax1.set(xlabel=r'Zeit $t$ [s]', ylabel=r'$T_E$ [°C]', title=r'Eistemperatur im Verlauf der Zeit')
ax1.legend()
ax1.grid()

fig.savefig("kal.svg")

print(am(temp_w_k))
print(stu(temp_w_k))
print(am(temp_e_k))
print(stu(temp_e_k))

In [ ]:

time_w_1, temp_w_1  = txtToArr('6.2_Wasser1.txt')


fig, ax = plt.subplots(figsize=(12,5))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.errorbar(time_w_1, temp_w_1, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label='Messwerte der Temperatur im Abstand von $1$s')

x_1 = np.linspace(0, max(time_w_1)+1, 100)
y_1 = np.full( shape=100, fill_value=am(temp_w_1))
ax.plot(x_1, y_1, color='#073b4c', label='Durchschnittstemperatur $\overline{T}_W$')

ax.set(xlabel=r'Zeit $t$ [s]', ylabel=r'Temperatur $T_W$ [°C]', title=r'Wassertemperatur im Verlauf der Zeit')
ax.legend()
ax.grid()

fig.savefig("Wasser_temp_1.svg")

print(am(temp_w_1))
print(stu(temp_w_1))

In [ ]:
time_w_2, temp_w_2  = txtToArr('6.2_Wasser1_2.txt')


fig, ax = plt.subplots(figsize=(12,5))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.errorbar(time_w_2, temp_w_2, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label='Messwerte der Temperatur im Abstand von $50$ms')

x_2 = np.linspace(0, max(time_w_2)+0.1, 100)
y_2 = np.full( shape=100, fill_value=am(temp_w_2))
ax.plot(x_2, y_2, color='#073b4c', label='Durchschnittstemperatur $\overline{T}_W$')

ax.set(xlabel=r'Zeit $t$ [s]', ylabel=r'Temperatur $T_W$ [°C]', title=r'Wassertemperatur im Verlauf der Zeit')
ax.legend()
ax.grid()

fig.savefig("Wasser_temp_2.svg")

print(am(temp_w_2))
print(stu(temp_w_2))

In [ ]:
time_er_1, temp_er_1  = txtToArr('6.2_Ergebnis.txt')


fig, ax = plt.subplots(figsize=(12,5))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.errorbar(time_er_1, temp_er_1, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label='Messwerte der Temperatur im Abstand von $50$ms')


ax.set(xlabel=r'Zeit $t$ [s]', ylabel=r'Temperatur $T_W$ [°C]', title=r'Wassertemperatur im Verlauf der Zeit')
ax.legend()
ax.grid()

fig.savefig("Erg_temp_1.svg")

In [ ]:
time_w_3, temp_w_3  = txtToArr('6.2_Wasser2.txt')


fig, ax = plt.subplots(figsize=(12,5))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.errorbar(time_w_3, temp_w_3, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label='Messwerte der Temperatur im Abstand von $1$s')

x_3 = np.linspace(0, max(time_w_3)+1, 100)
y_3 = np.full( shape=100, fill_value=am(temp_w_3))
ax.plot(x_3, y_3, color='#073b4c', label='Durchschnittstemperatur $\overline{T}_W$')

ax.set(xlabel=r'Zeit $t$ [s]', ylabel=r'Temperatur $T_W$ [°C]', title=r'Wassertemperatur im Verlauf der Zeit')
ax.legend()
ax.grid()

fig.savefig("Wasser_temp_3.svg")

print(am(temp_w_3))
print(stu(temp_w_3))

In [ ]:
time_w_4, temp_w_4  = txtToArr('6.2_Wasser2_2.txt')


fig, ax = plt.subplots(figsize=(12,5))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.errorbar(time_w_4, temp_w_4, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label='Messwerte der Temperatur im Abstand von $50$ms')

x_4 = np.linspace(0, max(time_w_4)+0.1, 100)
y_4 = np.full( shape=100, fill_value=am(temp_w_4))
ax.plot(x_4, y_4, color='#073b4c', label='Durchschnittstemperatur $\overline{T}_W$')

ax.set(xlabel=r'Zeit $t$ [s]', ylabel=r'Temperatur $T_W$ [°C]', title=r'Wassertemperatur im Verlauf der Zeit')
ax.legend()
ax.grid()

fig.savefig("Wasser_temp_4.svg")

print(am(temp_w_4))
print(stu(temp_w_4))

In [ ]:
time_er_2, temp_er_2  = txtToArr('6.2_Ergebnis_2.txt')


fig, ax = plt.subplots(figsize=(12,5))
ax.tick_params(axis='both', which='major', labelsize=18)
ax.tick_params(axis='both', which='minor', labelsize=16)

ax.errorbar(time_er_2, temp_er_2, xerr=time_f, yerr=temp_f, fmt='.', color='#118ab2', label='Messwerte der Temperatur im Abstand von $50$ms')


ax.set(xlabel=r'Zeit $t$ [s]', ylabel=r'Temperatur $T_W$ [°C]', title=r'Wassertemperatur im Verlauf der Zeit')
ax.legend()
ax.grid()

fig.savefig("Erg_temp_w.svg")

In [8]:
A1 = np.array([0.8 , 2.0 , 3.0 , 4.0 , 5.0 , 6.0 , 7.0 , 8.0 , 9.0 , 10.0 , 11.0 , 12.0 , 13.0 , 14.0 , 15.0 , 16.0 , 17.0 , 18.0 , 19.0 , 20.0 , 21.0 , 22.0 , 23.0 , 24.0 , 25.0 , 26.0 , 27.0 , 28.0 , 29.0 , 30.0 , 31.0 , 32.0 , 33.0 , 34.0 , 35.0 , 36.0 , 37.0 , 38.0 , 39.0 , 40.0 , 41.0 , 42.0 , 43.0 , 44.0 , 45.0 , 46.0 , 47.0 , 48.0 , 49.0 , 50.0 , 51.0 , 52.0 , 53.0 , 54.0 , 55.0 , 56.0 , 57.0 , 58.0 , 59.0 , 60.0 , 61.0 , 62.0 ])
A2 = np.array([102.1 , 103.6 , 103.4 , 103.3 , 103.6 , 97.3 , 103.6 , 103.6 , 103.5 , 103.7 , 103.1 , 103.2 , 103.4 , 103.4 , 103.1 , 103.5 , 103.9 , 104. , 104.9 , 104. , 104.1 , 103.8 , 104. , 103.9 , 103.7 , 104.3 , 104.3 , 104.6 , 104.2 , 104.4 , 104.2 , 104.1 , 104.1 , 104. , 104.3 , 103.6 , 103.9 , 103.9 , 103.7 , 103.6 , 97.7 , 104.2 , 103.3 , 103.9 , 104.2 , 103.6 , 103.9 , 103.8 , 104.2 , 104.2 , 103.9 , 105.3 , 105.6 , 105.1 , 104.4 , 105.1 , 104.1 , 104.5 , 103.9 , 104.2 , 104.2])
print(len(A1))
print(len(A2))

for i in range(len(A1)):
    print(i, A1[i], A2[i])

62
61
0 0.8 102.1
1 2.0 103.6
2 3.0 103.4
3 4.0 103.3
4 5.0 103.6
5 6.0 97.3
6 7.0 103.6
7 8.0 103.6
8 9.0 103.5
9 10.0 103.7
10 11.0 103.1
11 12.0 103.2
12 13.0 103.4
13 14.0 103.4
14 15.0 103.1
15 16.0 103.5
16 17.0 103.9
17 18.0 104.0
18 19.0 104.9
19 20.0 104.0
20 21.0 104.1
21 22.0 103.8
22 23.0 104.0
23 24.0 103.9
24 25.0 103.7
25 26.0 104.3
26 27.0 104.3
27 28.0 104.6
28 29.0 104.2
29 30.0 104.4
30 31.0 104.2
31 32.0 104.1
32 33.0 104.1
33 34.0 104.0
34 35.0 104.3
35 36.0 103.6
36 37.0 103.9
37 38.0 103.9
38 39.0 103.7
39 40.0 103.6
40 41.0 97.7
41 42.0 104.2
42 43.0 103.3
43 44.0 103.9
44 45.0 104.2
45 46.0 103.6
46 47.0 103.9
47 48.0 103.8
48 49.0 104.2
49 50.0 104.2
50 51.0 103.9
51 52.0 105.3
52 53.0 105.6
53 54.0 105.1
54 55.0 104.4
55 56.0 105.1
56 57.0 104.1
57 58.0 104.5
58 59.0 103.9
59 60.0 104.2
60 61.0 104.2


IndexError: index 61 is out of bounds for axis 0 with size 61

In [53]:
c_W, m_k, T_M, T_k, T_h, m_h = symbols('c_W m_k T_M T_k T_h m_h')
fuk = -c_W * (m_k * (T_M-T_k)/(T_h-T_M) - m_h)

c_W_w, c_W_f = 4.183 , 0.001
m_k_w, m_k_f = 100, 5
T_M_w, T_M_f = 53.33,0.07
T_k_w, T_k_f = 26.83, 0.05
T_h_w, T_h_f = 96.28, 0.04
m_h_w, m_h_f = 100, 5

werte = np.array([c_W_w,  m_k_w,  T_M_w,  T_k_w,  T_h_w,  m_h_w ])
unsic = np.array([c_W_f,  m_k_f,  T_M_f,  T_k_f,  T_h_f,  m_h_f ])
varia = np.array([c_W, m_k, T_M, T_k, T_h, m_h])

print("Der Wert beträgt:", fuk.subs({c_W:c_W_w, m_k:m_k_w, T_M:T_M_w, T_k:T_k_w, T_h:T_h_w, m_h:m_h_w}))
print("Der magische Fehler beträgt:", fehler(fuk, werte, unsic, varia)) 
print("Den größten Anteil am Fehler hat:", groessterBeitrag(fuk, werte, unsic, varia),"Das muss /1000")


fuk

Der Wert beträgt: 160.210360884750
Der magische Fehler beträgt: 24.6063913807462
Den größten Anteil am Fehler hat: (m_h, 20.9150000000000) Das muss /1000


-c_W*(-m_h + m_k*(T_M - T_k)/(-T_M + T_h))

In [50]:
#Fehler
m_W, m_Eis, gamma_K, T_W, T_M, T_Eis, c_W = symbols('m_W m_Eis gamma_K T_W T_M T_Eis c_W')
fuk = (m_W*c_W + gamma_K)*(T_W-T_M)/m_Eis - c_W*(T_M-T_Eis)

m_W_w, m_W_f = 200,5
m_Eis_w, m_Eis_f = 36,1
gamma_K_w, gamma_K_f = 160,25
T_W_w, T_W_f = 47.74,0.05
T_M_w, T_M_f = 22.396,0.017
T_Eis_w, T_Eis_f = 4.33,0.02
c_W_w, c_W_f = 4183/1000,1/1000

werte = np.array([m_W_w, m_Eis_w, gamma_K_w, T_W_w, T_M_w, T_Eis_w, c_W_w])
unsic = np.array([m_W_f, m_Eis_f, gamma_K_f, T_W_f, T_M_f, T_Eis_f, c_W_f])
varia = np.array([m_W, m_Eis, gamma_K, T_W, T_M, T_Eis, c_W])

print('Der Wert beträgt:', fuk.subs({m_W:m_W_w,m_Eis:m_Eis_w,gamma_K:gamma_K_w, T_W:T_W_w, T_M:T_M_w, T_Eis:T_Eis_w, c_W:c_W_w }))
print("Der Fehler ist:", fehler(fuk, werte, unsic, varia))
print("Der Größte Anteil ist:", groessterBeitrag(fuk, werte, unsic, varia))


Der Wert beträgt: 626.036322000000
Der Fehler ist: 30.1432588832010
Der Größte Anteil ist: (m_Eis, 19.4890666666667)
